# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=TransactionScout.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 TransactionScount

This notebook tries to show details of historical transactions.

It fetches the data from Solana, parses it, and then prints it.


In [ ]:
import base58
import datetime
import logging
import typing

logging.getLogger().setLevel(logging.ERROR)

from decimal import Decimal
from solana.publickey import PublicKey

from BaseModel import InstructionType, OwnedTokenValue, TokenLookup, TokenValue
from Context import Context, default_context
from Layouts import InstructionParsersByVariant, MANGO_INSTRUCTION_VARIANT_FINDER


# TransactionScout class

In [ ]:
class TransactionScout:
    def __init__(self, timestamp: datetime.datetime, signatures: typing.List[str],
                 accounts: typing.List[PublicKey], instructions: typing.List[typing.Any],
                 messages: typing.List[str], pre_token_balances: typing.List[OwnedTokenValue],
                 post_token_balances: typing.List[OwnedTokenValue]):
        self.timestamp: datetime.datetime = timestamp
        self.signatures: typing.List[str] = signatures
        self.accounts: typing.List[PublicKey] = accounts
        self.instructions: typing.List[typing.Any] = instructions
        self.messages: typing.List[str] = messages
        self.pre_token_balances: typing.List[OwnedTokenValue] = pre_token_balances
        self.post_token_balances: typing.List[OwnedTokenValue] = post_token_balances

    @property
    def summary(self) -> str:
        instructions = ", ".join([InstructionType(int(ins.variant)).name for ins in self.instructions])
        return f"« TransactionScout [{self.timestamp}] {instructions}\n    {self.signatures} »"

    def has_any_instruction_of_type(self, instruction_type: InstructionType) -> bool:
        return any(map(lambda ins: ins.variant == int(instruction_type), self.instructions))

    def _instruction_renderer(self, instruction) -> str:
        instruction_type = InstructionType(int(instruction.variant))
        additional_data = ""
        if instruction_type == InstructionType.InitMangoGroup:
            pass
        elif instruction_type == InstructionType.InitMarginAccount:
            pass
        elif instruction_type == InstructionType.Deposit:
            additional_data = f"quantity: {instruction.quantity}"
        elif instruction_type == InstructionType.Withdraw:
            additional_data = f"quantity: {instruction.quantity}"
        elif instruction_type == InstructionType.Borrow:
            additional_data = f"quantity: {instruction.quantity}, token index: {instruction.token_index}"
        elif instruction_type == InstructionType.SettleBorrow:
            additional_data = f"quantity: {instruction.quantity}, token index: {instruction.token_index}"
        elif instruction_type == InstructionType.Liquidate:
            additional_data = f"deposit quantities: {instruction.deposit_quantities}"
        elif instruction_type == InstructionType.DepositSrm:
            additional_data = f"quantity: {instruction.quantity}"
        elif instruction_type == InstructionType.WithdrawSrm:
            additional_data = f"quantity: {instruction.quantity}"
        elif instruction_type == InstructionType.PlaceOrder:
            pass
        elif instruction_type == InstructionType.SettleFunds:
            pass
        elif instruction_type == InstructionType.CancelOrder:
            pass
        elif instruction_type == InstructionType.CancelOrderByClientId:
            additional_data = f"client ID: {instruction.client_id}"
        elif instruction_type == InstructionType.ChangeBorrowLimit:
            additional_data = f"borrow limit: {instruction.borrow_limit}, token index: {instruction.token_index}"
        elif instruction_type == InstructionType.PlaceAndSettle:
            pass
        elif instruction_type == InstructionType.ForceCancelOrders:
            additional_data = f"limit: {instruction.limit}"
        elif instruction_type == InstructionType.PartialLiquidate:
            additional_data = f"max deposit: {instruction.max_deposit}"

        parameters = additional_data or "None"
        return f"« Instruction [{instruction_type.name}]: {parameters} »"

    @staticmethod
    def load_if_available(context: Context, signature: str) -> typing.Optional["TransactionScout"]:
        transaction_response = context.client.get_confirmed_transaction(signature)
        transaction_details = context.unwrap_or_raise_exception(transaction_response)
        if transaction_details is None:
            return None
        return TransactionScout.from_transaction_response(context, transaction_details)

    @staticmethod
    def load(context: Context, signature: str) -> "TransactionScout":
        tx = TransactionScout.load_if_available(context, signature)
        if tx is None:
            raise Exception(f"Transaction '{signature}' not found.")
        return tx

    @staticmethod
    def from_transaction_response(context: Context, response) -> "TransactionScout":
        def instruction_parser(data: str) -> typing.Tuple[InstructionType, typing.Any]:
            decoded = base58.b58decode(data)
            initial = MANGO_INSTRUCTION_VARIANT_FINDER.parse(decoded)
            parser = InstructionParsersByVariant[initial.variant]
            return parser.parse(decoded)

        def balance_to_token_value(accounts: typing.List[PublicKey], balance: typing.Dict) -> OwnedTokenValue:
            mint = PublicKey(balance["mint"])
            account = accounts[balance["accountIndex"]]
            amount = Decimal(balance["uiTokenAmount"]["amount"])
            decimals = Decimal(balance["uiTokenAmount"]["decimals"])
            divisor = Decimal(10) ** decimals
            value = amount / divisor
            token = TokenLookup.find_by_mint(context, mint)
            return OwnedTokenValue(account, TokenValue(token, value))

        try:
            instructions = []
            for instruction in response["transaction"]["message"]["instructions"]:
                parsed = instruction_parser(instruction["data"])
                instructions += [parsed]
            timestamp = datetime.datetime.fromtimestamp(response["blockTime"])
            accounts = list(map(PublicKey, response["transaction"]["message"]["accountKeys"]))
            signatures = response["transaction"]["signatures"]
            messages = response["meta"]["logMessages"]
            pre_token_balances = list(map(lambda bal: balance_to_token_value(accounts, bal), response["meta"]["preTokenBalances"]))
            post_token_balances = list(map(lambda bal: balance_to_token_value(accounts, bal), response["meta"]["postTokenBalances"]))
            return TransactionScout(timestamp,
                                    signatures,
                                    accounts,
                                    instructions,
                                    messages,
                                    pre_token_balances,
                                    post_token_balances)
        except Exception as exception:
            signature = "Unknown"
            if response and ("transaction" in response) and ("signatures" in response["transaction"]) and len(response["transaction"]["signatures"]) > 0:
                signature = ", ".join(response["transaction"]["signatures"])
            print(response)
            raise Exception(f"Exception fetching transaction '{signature}'", exception)

    def __str__(self) -> str:
        def format_tokens(account_token_values: typing.List[OwnedTokenValue]) -> str:
            if len(account_token_values) == 0:
                return "\n        None"
            return "\n        ".join([f"{atv}" for atv in account_token_values])

        instruction_names = ", ".join([InstructionType(int(ins.variant)).name for ins in self.instructions])
        signatures = "\n        ".join(self.signatures)
        accounts = "\n        ".join([f"{acc}" for acc in self.accounts])
        messages = "\n        ".join(self.messages)
        instructions = "\n        ".join(map(self._instruction_renderer, self.instructions))
        changes = OwnedTokenValue.changes(self.pre_token_balances, self.post_token_balances)
        tokens_in = format_tokens(self.pre_token_balances)
        tokens_out = format_tokens(self.post_token_balances)
        token_changes = format_tokens(changes)
        return f"""« TransactionScout {self.timestamp}: {instruction_names}
    Signatures:
        {signatures}
    Instructions:
        {instructions}
    Accounts:
        {accounts}
    Messages:
        {messages}
    Tokens In:
        {tokens_in}
    Tokens Out:
        {tokens_out}
    Token Changes:
        {token_changes}
»"""

    def __repr__(self) -> str:
        return f"{self}"


In [ ]:
def fetch_all_todays_transaction_signatures() -> typing.List[str]:
    now = datetime.datetime.now()
    one_day_ago = now - datetime.timedelta(days=1)
    one_day_ago_timestamp = one_day_ago.timestamp()

    all_fetched = False
    before = None
    signature_results = []
    while not all_fetched:
        signature_response = default_context.client.get_confirmed_signature_for_address2(default_context.group_id, before=before)
        signature_result = default_context.unwrap_or_raise_exception(signature_response)
        signature_results += signature_result
        if (len(signature_result) == 0) or (signature_result[-1]["blockTime"] < one_day_ago_timestamp):
            all_fetched = True
        before = signature_results[-1]["signature"]

    todays = [res for res in signature_results if res["blockTime"] > one_day_ago_timestamp]
    todays_signatures = list([res["signature"] for res in todays])
    return todays_signatures


# 🏃 Running

You can run the following cells to examine any Mango transaction.

Enter the signature of the transaction you want to examine in the value for `TRANSACTION_TO_VIEW` in the box below, between the double-quote marks. Then run the notebook by choosing 'Run > Run All Cells' from the notebook menu at the top of the page.

Alternatively you can uncomment the `rx` code to run through all recent Mango Solana transactions, printing a summary or (if it's a `PartialLiquidate` transaction) the full details.

**NOTE**: Transactions disappear from most servers within a few hours. It is often not possible to go too far back searching for transactions or retrieving their data.

In [ ]:
TRANSACTION_TO_VIEW = ""

In [ ]:
if __name__ == "__main__":
    if TRANSACTION_TO_VIEW != "":
        print(TransactionScout.load(default_context, TRANSACTION_TO_VIEW))

#     import rx
#     import rx.operators as ops
#     signatures = fetch_all_todays_transaction_signatures()
#     rx.from_(signatures).pipe(
#         ops.map(lambda signature: TransactionScout.load_if_available(default_context, signature)),
#         ops.filter(lambda item: item is not None),
#     #     ops.filter(lambda item: item.has_any_instruction_of_type(InstructionType.PartialLiquidate)),
#     #     ops.take(100),
#         ops.map(lambda item: print(f"{item}") if item.has_any_instruction_of_type(InstructionType.PartialLiquidate) else print(f"{item.summary}"))
#     ).run()
